In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from transformers import pipeline
tokenizer = AutoTokenizer.from_pretrained("d4data/bias-detection-model")
model = TFAutoModelForSequenceClassification.from_pretrained("d4data/bias-detection-model")

classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at d4data/bias-detection-model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [3]:
def custom_classification(x):
    classi_out = classify(x)
    return classi_out[0]['label'], classi_out[0]['score']

def custom_recognizer(x):
    biased_words = recognizer(x)
    biased_words_list = []
    for id in range(0, len(biased_words)):
        biased_words_list.append(biased_words[id]['entity'])
    return ", ".join(biased_words_list)

def custom_debiasing(x):
    suggestions = run(x)
    if suggestions == None:
        return ""
    else:
      all_suggestions = []
      for sent in suggestions[0:3]:
        all_suggestions.append(sent['Sentence'])
      return "\n\n".join(all_suggestions)

In [5]:
# Load gender bias headlines

headlines = pd.read_csv("gender_bias_data.csv")

print(headlines.columns)

Index(['headline_no_site'], dtype='object')


In [7]:
headlines['prediction'] = headlines['headline_no_site'].apply(classifier)

# Extract the label and score from the prediction
headlines['state'] = headlines['prediction'].apply(lambda x: x[0]['label'])
headlines['score'] = headlines['prediction'].apply(lambda x: x[0]['score'])

# Drop the 'prediction' column if you don't need it anymore
headlines = headlines.drop(columns=['prediction'])

# Print the resulting DataFrame with bias predictions
print(headlines[['headline_no_site', 'state', 'score']])



                                     headline_no_site       state     score
0   Life sentence for Plymouth woman who murdered ...  Non-biased  0.980371
1   “I Had Never Even Heard Of Her Before She Cure...  Non-biased  0.681668
2   'Honey Boo Boo' Mom Wants Female Pres as Hilla...      Biased  0.892100
3   New York Woman Says Rihanna's Lipstick Gave He...      Biased  0.781223
4   Teens Notice Pretty Blonde Woman In Car Beside...      Biased  0.992805
5   Crime Scene   Mother charged after Pr. William...  Non-biased  0.873353
6   She Had The Meanest Mom In The World. Now? She...      Biased  0.990178
7   A US woman on why she disagrees with Obama on ...      Biased  0.900038
8   EastEnders star Emer Kenny’s sister told she w...      Biased  0.533608
9   Army Colonel: Use 'Average Looking Women' to P...      Biased  0.639923
10  Size 30 woman ditches cruel husband who called...      Biased  0.969968
11  Mom Hopes For A Normal Baby, But When Doctors ...      Biased  0.995048
12  Watch th

In [8]:
headlines.to_csv('headlines_predictions.csv', index=False)